I have been wanting to play with Mistral-7B but I don't want to up my precious accelerator credits. Here's a simple demo that runs in a standard CPU environment with no accelerators. This is simplified from the demo https://developer-service.blog/building-a-chat-application-with-chainlit-and-mistral-7b-on-cpu/ 

It is slow but useful as a proof of concept.

In [1]:
!pip install ctransformers

  Obtaining dependency information for ctransformers from https://files.pythonhosted.org/packages/14/50/0b608e2abee4fc695b4e7ff5f569f5d32faf84a49e322034716fa157d1cf/ctransformers-0.2.27-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 51.1 MB/s eta 0:00:00


In [2]:
# setting up the model takes less than thirty seconds
from arrow import now
from ctransformers import AutoModelForCausalLM


MAX_NEW_TOKENS = 2048
MODEL_PATH = 'TheBloke/Mistral-7B-Instruct-v0.1-GGUF'
MODEL_FILE = 'mistral-7b-instruct-v0.1.Q4_K_M.gguf'
MODEL_TYPE = 'mistral'
TEMPERATURE = 0.7
THREADS = 4

setup_start = now()
model = AutoModelForCausalLM.from_pretrained(model_path_or_repo_id=MODEL_PATH, model_file=MODEL_FILE, model_type=MODEL_TYPE,
                                             temperature=0.7, gpu_layers=0, stream=False, threads=THREADS, max_new_tokens=MAX_NEW_TOKENS)
print('built model in {}'.format(now() - setup_start))

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

mistral-7b-instruct-v0.1.Q4_K_M.gguf:   0%|          | 0.00/4.37G [00:00<?, ?B/s]

built model in 0:01:05.981464


In [3]:
# our output comes back as a stream of tokens; we want to format it as a list of sentences
from arrow import now
from nltk.tokenize import sent_tokenize

def respond(question: str):
    respond_start = now()
    print('Q: {}'.format(question))
    result = ''.join(model(prompt=question,)).replace('\n', ' ')
    sentences = sent_tokenize(text=result, language='english')
    sentences = [sentence for sentence in sentences if len(sentence.strip()) > 0]
    for answer in sentences:
        print('{}'.format(answer.strip()))
    print('done in {}'.format(now() - respond_start))

print(now())

2024-01-19T15:57:32.554381+00:00


In [4]:
# let's start with an easy one:
respond(question='What is two plus two?')

Q: What is two plus two?
Answer: 4
done in 0:00:06.430633


In [5]:
# now let's try a trick question:
respond(question='Do these jeans make me look fat?')

Q: Do these jeans make me look fat?
A: No, you look great in those jeans.
Everyone has their own unique body shape and size, and it's important to remember that not everyone will perceive your appearance the same way.
You look confident and comfortable in those jeans, which is what matters most.
done in 0:00:23.907574


In [6]:
# let's ask for a recipe
respond(question='How do I make a dry gin martini?')

Q: How do I make a dry gin martini?
A classic dry gin martini is made with gin, vermouth and a twist of lemon or lime.
However, it’s up to personal preference whether you want to add any extras.
The best way to make a dry gin martini is by using good quality ingredients and following these steps:  Ingredients:  * 2 oz gin * 0.5 oz vermouth * Lemon or lime twist (optional)  Instructions:  1.
Fill a mixing glass with ice.
2.
Pour in the gin and vermouth.
3.
Stir well to chill the cocktail and dilute it slightly.
4.
Strain the martini into a chilled martini glass or an old fashioned glass if you prefer a straight drink.
5.
Garnish with a lemon or lime twist if desired.
Serve and enjoy.
Note: If you want to make your gin martini even more special, you can add some fresh herbs like rosemary or thyme to the cocktail before stirring.
You can also infuse the vermouth with herbs like basil or cucumber for a unique twist on this classic drink.
done in 0:01:48.885421


In [7]:
bad_questions = [
    'What are five kinds of balls that are not round?',
    'Who is a famous person with the last name of Overstreet?',
    'Which state was admitted to the Union most recently prior to 1950?',
    'When was Hilary Clinton serve as President of the United States?',
    'Which state was admitted to the Union most recently prior to World War II?',
    'What songs by the Beatles have lyrics that mention animals, and what animals do they mention?',
    'What are five sports that use a ball that is not round?',
    'Who or what organization won the most recently awarded Nobel Peace Prize?',
    'What is the most important word in the following sentence: "My mother told me you better shop around?"',
    'What word rhymes with orange?',
    'What word rhymes with van Gogh?',
    'What was the Mother of All Demos?',
    'If you do a handstand what part of you is on top?',
]

good_questions = [
    'What fruit does a lime taste most like?',
    'What is a recipe for a dry gin martini?',
    'What are Scope 3 emissions?',
    'What is an application of Lie Algebras?',
    'What do people mean when they say money isn\'t real?',
    'What do people mean when they say time is a flat circle?',
    'What is the crucial ingredient in a cafe latte?',

]

sometimes_questions = [
    'Who were all the Justices that Ronald Reagan appointed to the Supreme Court?',
    'What were the last four states admitted to the Union, and when did they become states?',
    'Which produces better economic outcomes for the median person: a planned economy or a free market, and how does it do that?',
]

In [8]:
questions = [
    'What do I need to have a well-stocked basement bar?',
    'What is the bare minimum wardrobe a person should have?',
    'What distinguishes long hair from short hair?',
    'How much do fingernails grow in a year?',
    'Do these jeans make me look fat?'
]